### 0. Imports and pre-process required files

In [ ]:
# requirements installation (for Jupyter)
!{sys.executable} -m pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org selenium
!{sys.executable} -m pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org tkinter
!{sys.executable} -m pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org pickle5
!{sys.executable} -m pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org pandas
!{sys.executable} -m pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org numpy
!{sys.executable} -m pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org wakepy
!{sys.executable} -m pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org investpy
!{sys.executable} -m pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org countrynames
!{sys.executable} -m pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org ccy
!{sys.executable} -m pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org pyodbc

In [ ]:
# requirements installation (for Windows)

import subprocess

subprocess.run(f'pip config set global.trusted-host "pypi.org files.pythonhosted.org pypi.python.org"', shell=True)

dependencies = ['selenium', 'webdriver-manager', 'chromium-chromedriver',
                'tkinter', 'pickle5', 'pandas', 'numpy', 'wakepy', 'investpy',
                'countrynames', 'ccy', 'pyodbc', 'openpyxl']

for dependency in dependencies:
    # For bank's server (linux-based)
    # command = f'{sys.executable} -m pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org {dependency}'
    
    # For localhost (windows)
    command = f'pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org {dependency}'
    
    subprocess.run(command, shell=True)

In [ ]:
# Manual PIA import (Hive) to pia_imported.csv (./input)
%run -i ./system_modules/pia_prep.py

In [ ]:
# Manual BR.xlsx (./input) to BR_N.csv (./source_files)
%run -i ./system_modules/br_prep.py

In [ ]:
# Manual SR (DSR, ESR, FFR) import from Hive to SR_N.csv (./source_files)
from system_modules.sys_class import *
try:
    SR_cache = cache(name='SR', data=pd.read_csv('./source_files/SR_N.csv'))
    SR_cache.path = './system_modules/SR.cache'
    SR_cache.cached()
except:
    pass

%run -i ./system_modules/sr_prep.py

os.remove(SR_cache.path)
os.remove('./system_modules/SR.cache')

In [ ]:
# Manual SR (ISR) import from Hive to ISR_N.csv (./source_files)
from system_modules.sys_class import *
try:
    ISR_cache = cache(name='ISR', data=pd.read_csv('./source_files/ISR_N.csv'))
    ISR_cache.path = './system_modules/ISR.cache'
    ISR_cache.cached()
except:
    pass

%run -i ./system_modules/isr_prep.py

os.remove(ISR_cache.path)
os.remove('./system_modules/ISR.cache')

### 1. Define and imports

In [ ]:
# Imports and defines

import sys
import os

sys.path.append(os.getcwd()+'/system_modules/')

from system_modules.imports import *
from system_modules.sys_functions import *
from system_modules.sys_class import *
from system_modules.sys_security import *
from system_modules.sys_investor import *

try:
    # Load input file and generate/read cache
    TMP_PIA_DF_NAME, TMP_PIA_DF = df_finds(keyword='pia', requiredName=True)
    NEW_PIA_DF = DataFrameFile(name = TMP_PIA_DF_NAME, dataframe = TMP_PIA_DF)

    # Generate empty tables or read if exists
    YEARLY_PIA_DF = DataFrameFile(name=f'YEARLY_PIA_DF_{datetime.today().year}',
                                  dataframe=df_read(name=f'YEARLY_PIA_DF_{datetime.today().year}',
                                                    clone=pd.DataFrame(columns=['msr_prd_id', 'end_ivstr_unq_id_tp', 'pvr_role_rspn', 
                                                                                'org_ip_id', 'RID', 'end_ivstr_nm',
                                                                                'SECTOR', 'ISIC4_CLID','orig_ccy_cost_val',
                                                                                'orig_ccy_mkt_val', 'scr_code', 'BOP_item',
                                                                                'issuer_name', 'issuer_countryID',
                                                                                'scr_currencyID', 'scr_termID', 'scr_instrument'
                                                                                ])))
    
    INVESTOR_TABLE = DataFrameFile(name='INVESTOR_TABLE',
                                  dataframe=df_read(name='INVESTOR_TABLE',
                                                   clone=pd.DataFrame(columns=['RID','end_ivstr_unq_id_tp',
                                                                               'end_ivstr_nm','pvr_role_rspn',
                                                                               'org_ip_id', 'SECTOR','ISIC4_CLID'
                                                                               ])))
    
    SECURITY_TABLE = DataFrameFile(name='SECURITY_TABLE',
                                  dataframe=df_read(name='SECURITY_TABLE',
                                                   clone=pd.DataFrame(columns=['scr_code', 'BOP_item', 'issuer_name',
                                                                               'issuer_countryID', 'scr_currencyID',
                                                                               'scr_termID', 'scr_instrument'])))
    
    NEW_PIA_CACHE = cache(name=TMP_PIA_DF_NAME)
    
except:
    traceback_log()
    print('Error occured')
    
else:
    finished(f'start.ipynb (Imports and defines), file name: {TMP_PIA_DF_NAME}, total data rows to-be processed is {len(NEW_PIA_DF.dataframe)}')

### 2.1 End Investor Process
- BR data mapping

##### 2.1.1 Normal Investor Mapping

In [ ]:
# End Investor Process
from system_modules.sys_investor import *

try:
    print(f"start.ipynb (Investor Process): pre-process found row counts is {len(INVESTOR_TABLE.dataframe)} {datetime.now()}")
    INVESTOR_TABLE = investor_validation(NEW_PIA_DF, INVESTOR_TABLE)
    print(f"\nstart.ipynb (Investor Process): post-process found row counts is {len(INVESTOR_TABLE.dataframe)} {datetime.now()}")
    
    INVESTOR_TABLE.dataframe = INVESTOR_TABLE.dataframe.applymap(lambda x: str(x).replace(".0", ""))
    INVESTOR_TABLE.dataframe['RID'] = INVESTOR_TABLE.dataframe['RID'].astype(str).apply(lambda x: x.zfill(13)).replace(['nan', '0000000000nan', '000000000None'], np.nan)
    INVESTOR_TABLE.dataframe = INVESTOR_TABLE.dataframe.replace('nan', np.NaN)
    INVESTOR_TABLE.dataframe.sort_values("RID", inplace=True)
    INVESTOR_TABLE.export()
    
    # check for missing records in INV_NF
    INV_NF = pd.read_csv('./output/INV_NOT_FOUND.csv')
    if len(INV_NF) > 0:
        print('* Existing not found records, please fix the record and run cell 2.1.1')
        
except (KeyboardInterrupt):
    pass

except:
    traceback_log()
    print('Error occured')

else:
    finished(f'\nstart.ipynb (Investor Process) {datetime.now()}')
    INVESTOR_TABLE.dataframe

##### 2.1.2 Move from NOT FOUND back

In [ ]:
# 2.1.2 Re-add INV_NOT_FOUND data back to INV_TABLE (if fixed)
# this part of the code will attempt to move fixed records to INVESTOR_TABLE

# read NF files
process = False
try:
    if len(INV_NF) >= 0:
        process = True
except (FileExistsError, FileNotFoundError):
    print(f"start.ipynb (Investor Update Process): length of NOT FOUND file is 0, no update is required {datetime.now()}")
except:
    INV_NF = pd.read_excel('./output/INV_NOT_FOUND.xlsx')
    process = True
    
# process
if process:
    try:
        INV_NF.reset_index(drop=True, inplace=True)
        to_drop_lst = []
        if len(INV_NF) > 0:
            for row in range(len(INV_NF)): # for all rows in NF
                
                # if sector and isic is empty, go to next row
                if pd.isna(INV_NF.loc[row, 'SECTOR']) and pd.isna(INV_NF.loc[row, 'ISIC4_CLID']):
                    pass
                
                # else format sector/isic and add row to investor table
                else:
                    INV_NF.loc[row, 'SECTOR'] = str(INV_NF.iloc[row]['SECTOR']).replace(".0", "")
                    INV_NF.loc[row, 'ISIC4_CLID'] = str(INV_NF.iloc[row]['ISIC4_CLID']).replace(".0", "")
                    INVESTOR_TABLE.dataframe = pd.concat([INVESTOR_TABLE.dataframe, INV_NF.iloc[[row]]], ignore_index=True)
                    to_drop_lst.append(row)
            print(f"start.ipynb (Investor Update Process): INVESTOR TABLE updated, {len(to_drop_lst)} rows updated {datetime.now()}")
        
        else:
            print(f"start.ipynb (Investor Update Process): length of NOT FOUND file is 0, no update is required {datetime.now()}")           
        
        INV_NF = INV_NF.drop(index=to_drop_lst)
        INV_NF.reset_index(drop=True, inplace=True)
        INVESTOR_TABLE.dataframe.reset_index(drop=True, inplace=True)
        
    except (KeyboardInterrupt):
        pass

    except: 
        print('Error occured')
        traceback_log()
        
    else:
        INVESTOR_TABLE.dataframe = INVESTOR_TABLE.dataframe.applymap(lambda x: str(x).replace(".0", ""))
        INVESTOR_TABLE.dataframe['RID'] = INVESTOR_TABLE.dataframe['RID'].astype(str).apply(lambda x: x.zfill(13)).replace(['nan', '0000000000nan', '000000000None'], np.nan)
        INVESTOR_TABLE.dataframe = INVESTOR_TABLE.dataframe.replace('nan', np.NaN)
        INVESTOR_TABLE.dataframe.sort_values("RID", inplace=True)
        INVESTOR_TABLE.export()
        
        if len(INV_NF) > 0:
            INV_NF.to_excel("./output/INV_NOT_FOUND.xlsx", index=False)
        else:
            try:
                os.remove('./output/INV_NOT_FOUND.xlsx')
            except (FileNotFoundError):
                pass
            
else:
    try:
        os.remove('./output/INV_NOT_FOUND.xlsx')
    except (FileNotFoundError):
        pass

### 2.2 Security Process (optional)
- PIA data scraping
- SR (DSR/ESR/FFR) data mapping
- ISR data mapping
##### *Please use personal internet connection for this part

* Scraped around new 100 securities from over 1.6m rows, accounted for 0.00625% total rows required for processing

In [ ]:
# Security Process (Optional)
from system_modules.sys_security import *

try:
    print(f"start.ipynb (Security Process): pre-process row counts is {len(SECURITY_TABLE.dataframe)}, {datetime.now()}")
    SECURITY_TABLE = security_validation(NEW_PIA_DF, SECURITY_TABLE, self_recheck_override=False)
    print(f"start.ipynb (Security Process): pre-process row counts is {len(SECURITY_TABLE.dataframe)}, {datetime.now()}")
    SECURITY_TABLE.dataframe[['BOP_item', 'issuer_countryID', 'scr_currencyID', 'scr_termID']] = SECURITY_TABLE.dataframe[['BOP_item', 'issuer_countryID', 'scr_currencyID', 'scr_termID']].applymap(lambda x: str(x).replace(".0", ""))
    SECURITY_TABLE.dataframe = SECURITY_TABLE.dataframe.replace(['nan', '-'], np.NaN)# .replace('.0', np.NaN)
    SECURITY_TABLE.dataframe
    SECURITY_TABLE.export()


except (KeyboardInterrupt):
    pass

except (Exception):
    traceback_log()
    print('Error occured')

else:
    finished(f'\nstart.ipynb (Security Process) {datetime.now()}')

In [ ]:
SECURITY_TABLE.dataframe

### 3. PIA Cleansing (monthly process)
- SECURITY/INVESTOR TABLE mapping

In [ ]:
# PIA cleansing method 1 (faster, no cache, utilize concurrent/multiprocessing, can't continue where left off)

try:
    print(f'start.ipynb: PIA cleansing running {datetime.now()}')
    PIA_PROCESS_CACHE = cache(name='PIA_PROCESS_CACHE', data=[YEARLY_PIA_DF, NEW_PIA_DF, INVESTOR_TABLE, SECURITY_TABLE], force=True)
    print(f'start.ipynb: PIA cleansing: cached, starting cleansing {datetime.now()}')
    try:
        output_terminal = subprocess.run(["python", "./system_modules/sys_pia.py"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    except (FileNotFoundError):
        output_terminal = subprocess.run(["/storage-02/venv/local/nuntapoj/local_3a_env01/bin/python", "./system_modules/sys_pia.py"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print(f'start.ipynb: PIA cleansing finished {datetime.now()}')
    
    # import files back
    YEARLY_PIA_DF = PIA_PROCESS_CACHE.data[0]
    NEW_PIA_DF =  PIA_PROCESS_CACHE.data[1]
    INVESTOR_TABLE = PIA_PROCESS_CACHE.data[2]
    SECURITY_TABLE = PIA_PROCESS_CACHE.data[3]
    
    # Loop to read the output line by line
    for line in output_terminal.stdout.decode().splitlines():
        print(line)

    # Check if there were any errors
    if output_terminal.returncode != 0:
        print("Error:", output_terminal.stderr.decode())
    
except (KeyboardInterrupt):
    raise (KeyboardInterrupt)

except (Exception):
    traceback_log()
    print('Error occured')
    
else:
    # get all files in cache folder, then move current pia file from input to processed_input
    cache_files = get_all_files(path='./cache', keyword='pia')
    pia_to_move = [row.replace('.cache', '') if 'pia' in row else row for row in cache_files][0]
    shutil.move(f"./input/{pia_to_move}", f"./processed_input/{pia_to_move}")
    
    # clear cache
    clear_cache()
    
    finished(f'\nstart.ipynb (PIA Process) {datetime.now()}')
    

In [ ]:
YEARLY_PIA_DF.dataframe

In [ ]:
# NEW_PIA_DF_SHORTEN = NEW_PIA_DF
# NEW_PIA_DF_SHORTEN.dataframe = NEW_PIA_DF.dataframe.sample(5000)
# IA_PROCESS_CACHE = cache(name='PIA_PROCESS_CACHE', data=[YEARLY_FLOW_DF, YEARLY_PIA_DF, NEW_PIA_DF, INVESTOR_TABLE, SECURITY_TABLE], force=True)
# %run -i ./system_modules/sys_pia.py

# Test
Use case: If program cached current process records and got struck, use clear cache can reset current processed records

Note:
- Cache for PIA will cache the current processed row for PIA file in 3.
- Cache for INVESTOR/SECURITY will cache the current processed row for PIA file in 2., and also the file that it is processing (meaning there can be many cache files for SEC/INV, each saved separately)

In [ ]:
clear_cache()